In [2]:
import pandas as pd
import torch

In [3]:
df=pd.read_csv("data/tox21/raw/tox21.csv")
smiles_list=list(df["smiles"])
target = df[['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD',
           'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']]

target = target.replace(0, -1)  # convert 0 to -1
target = target.fillna(0)  

target = torch.tensor(target.values,dtype=torch.float)

In [4]:
target.shape

torch.Size([7831, 12])

In [7]:
target[1].unsqueeze(0).shape

torch.Size([1, 12])

In [17]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from tqdm import tqdm


In [18]:
symbol_list=[]
for i, smile in enumerate(tqdm(df["smiles"])):
    mol = AllChem.MolFromSmiles(smile)
    for atom in mol.GetAtoms():
        symb=atom.GetSymbol()
        if symb not in symbol_list:
            symbol_list.append(symb)

100%|██████████| 7831/7831 [00:02<00:00, 3114.94it/s]


In [26]:
strm=""
for i,symb in enumerate(symbol_list):
    st="\'"+symb+"\'"+":"+str(i)+","
    strm+=st

In [27]:
strm

"'C':0,'O':1,'N':2,'S':3,'P':4,'Cl':5,'I':6,'Zn':7,'F':8,'Ca':9,'As':10,'Br':11,'B':12,'H':13,'K':14,'Si':15,'Cu':16,'Mg':17,'Hg':18,'Cr':19,'Zr':20,'Sn':21,'Na':22,'Ba':23,'Au':24,'Pd':25,'Tl':26,'Fe':27,'Al':28,'Gd':29,'Ag':30,'Mo':31,'V':32,'Nd':33,'Co':34,'Yb':35,'Pb':36,'Sb':37,'In':38,'Li':39,'Ni':40,'Bi':41,'Cd':42,'Ti':43,'Se':44,'Dy':45,'Mn':46,'Sr':47,'Be':48,'Pt':49,'Ge':50,"